In [1]:
import numpy as np
import torch.nn as nn
import scipy as sp
import matplotlib.pyplot as plt
import math
import string
import re

In [2]:
import transformers #These two cells concern the construction of the object pipeline

In [3]:
source = "gilf/french-camembert-postag-model"

model = transformers.AutoModelForTokenClassification.from_pretrained(source)

In [11]:
tokenizer = transformers.AutoTokenizer.from_pretrained(source)

classifier = transformers.pipeline('ner', model=model, tokenizer=tokenizer, grouped_entities=True)

In [21]:
sampleText = "Le persan (autonyme : fārsī, ou pārsī) est une langue indo-européenne. C'est la langue officielle et majoritaire de l'Iran (centre, centre-sud, nord-est), de l'Afghanistan et du Tadjikistan. Le persan est une langue notable mais minoritaire à Bahreïn, en Irak, à Oman, au Qatar et aux Émirats arabes unis. Le dari, dialecte du persan, également appelé persan afghan ou persan oriental, est une langue officielle en Afghanistan ; il est parlé également en Iran et au Pakistan. De même, le tadjik, autre dialecte du persan, est la langue officielle du Tadjikistan et également parlé au Kirghizistan, au Turkménistan, en Ouzbékistan, et dans une moindre mesure au Kazakhstan. Le persan ou l’une de ses langues-sœurs est également parlé en Azerbaïdjan, en Russie et en Ouzbékistan (minorité tadjike). Jadis, du XVIe au XIXe siècle, il fut la langue officielle de l'Empire moghol. Le persan fait partie du groupe indo-iranien de la famille des langues indo-européennes. C'est une langue du type « sujet-objet-verbe ». Les langues persanes s’écrivent surtout au moyen de l'alphabet arabo-persan, variante de l'alphabet arabe, bien qu'elles n'aient aucune parenté avec la langue arabe, dont elles diffèrent tant sur le plan de la grammaire que de la phonologie. Au Tadjikistan, en Russie, en Azerbaïdjan et en Ouzbékistan, le tadjik s’écrit au moyen de l'alphabet cyrillique."

In [5]:
class Graph(object):
    def __init__(self, n, E, dir=False):
        self.n = n
        self.E = E
        self.score = [1 for i in range(n)]
        self.inV = [[] for i in range(n)]
        self.outV = [[] for i in range(n)]
        
        for e in E:
            w = 1 if len(e) == 2 else e[2]
            
            self.inV[e[1]].append([e[0], w])
            self.outV[e[0]].append([e[1], w])
            if not dir:
                self.inV[e[0]].append([e[1], w])
                self.outV[e[1]].append([e[0], w])
    
    def textRank(self, d=0.85):
        maxShift = 1
        
        while(maxShift >= 1e-4):
            newScore = [(1-d) for i in range(self.n)]
            for i in range(self.n):
                for l, j in enumerate(self.inV[i]):
                    newScore[i] += d * self.score[j[0]] * self.inV[i][l][1]/sum([k[1] for k in self.outV[j[0]]])
            maxShift = max([abs(self.score[k] - newScore[k]) for k in range(self.n)])
            self.score = newScore
        return self.score

In [6]:
g = Graph(4, [[0, 1], [0, 2], [0, 3], [1, 2], [2, 3], [3, 0]])

print(g.textRank())

[1.2961485736273217, 0.7107506361068179, 1.0068992097341396, 0.9862015805317207]


In [15]:
def tokenize(sentence, removeDup=False):
    rawData = classifier(sentence)
    cleanedData = [word for word in rawData if (word['entity_group'] in ['ADJ', 'ADJWH', 'ET', 'NC', 'NPP'])]
    tokens = [word['word'] for word in cleanedData]
    return (list(dict.fromkeys(tokens)) if removeDup else tokens)

def similarity(s1, s2): #Welp, not the most definite thing, but a good start to test stuff
    w1 = tokenize(s1, removeDup=True)
    w2 = tokenize(s2, removeDup=True)
    n = sum([(1 if k in w2 else 0) for k in w1])
    return n/(math.log(max(2, len(w1)))+math.log(max(2, len(w2))))

tokenize("Face à un choc inédit, les mesures mises en place par le gouvernement ont permis une protection forte et efficace des ménages")

['choc',
 'inédit',
 'mesures',
 'place',
 'gouvernement',
 'protection',
 'forte',
 'efficace',
 'ménages']

In [23]:
def sentenceRank(t): #Here, t stands for text, s for sentence, w for wordlist
    sl = re.split("(?<=[.?!])", t) #Sentence splitting
    sl = [s if (s == "" or s[-1] not in [" ", "\n"]) else s[:-1] for s in sl]
    sl = [s if (s == "" or s[0] not in [" ", "\n"]) else s[1:] for s in sl]
    sl = [s for s in sl if len(tokenize(s)) != 0] #Cleaning up
    
    n = len(sl) #Graph construction
    gs = []
    for i in range(n):
        for j in range(i): #The graph is undirected, we avoid duplicates
            sim = similarity(sl[i], sl[j])
            if sim != 0:
                gs.append([i, j, sim])
    g = Graph(n, gs, dir=False)
    return list(zip(sl, g.textRank()))

sentenceRank(sampleText)

[('Le persan (autonyme : fārsī, ou pārsī) est une langue indo-européenne.',
  1.0245149545900247),
 ("C'est la langue officielle et majoritaire de l'Iran (centre, centre-sud, nord-est), de l'Afghanistan et du Tadjikistan.",
  1.0550397737656654),
 ('Le persan est une langue notable mais minoritaire à Bahreïn, en Irak, à Oman, au Qatar et aux Émirats arabes unis.',
  1.110880874604189),
 ('Le dari, dialecte du persan, également appelé persan afghan ou persan oriental, est une langue officielle en Afghanistan ; il est parlé également en Iran et au Pakistan.',
  1.1550130159632974),
 ('De même, le tadjik, autre dialecte du persan, est la langue officielle du Tadjikistan et également parlé au Kirghizistan, au Turkménistan, en Ouzbékistan, et dans une moindre mesure au Kazakhstan.',
  1.4242415153739978),
 ('Le persan ou l’une de ses langues-sœurs est également parlé en Azerbaïdjan, en Russie et en Ouzbékistan (minorité tadjike).',
  0.8923216105273419),
 ("Jadis, du XVIe au XIXe siècle, il

In [38]:
def summarize(t, length):
    unsortedList = [(k[0], k[1], i) for i, k in enumerate(sentenceRank(t))]
    sortedList = sorted(unsortedList, key=(lambda k: -k[1])) #Sorting
    sentences = [s for i, s in enumerate(sortedList) if (i == 0 or s[0] != sortedList[i-1][0])] #Removing duplicates
    
    summarySentences = []
    l = 0
    for s in sentences:
        l += len(tokenize(s[0]))
        if(l > length):
            break
        summarySentences.append(s)
    summary = [s[0] for s in sorted(summarySentences, key=(lambda k: k[2]))]
    return " ".join(summary)

print(sampleText)
print(summarize(sampleText, 50))

Le persan (autonyme : fārsī, ou pārsī) est une langue indo-européenne. C'est la langue officielle et majoritaire de l'Iran (centre, centre-sud, nord-est), de l'Afghanistan et du Tadjikistan. Le persan est une langue notable mais minoritaire à Bahreïn, en Irak, à Oman, au Qatar et aux Émirats arabes unis. Le dari, dialecte du persan, également appelé persan afghan ou persan oriental, est une langue officielle en Afghanistan ; il est parlé également en Iran et au Pakistan. De même, le tadjik, autre dialecte du persan, est la langue officielle du Tadjikistan et également parlé au Kirghizistan, au Turkménistan, en Ouzbékistan, et dans une moindre mesure au Kazakhstan. Le persan ou l’une de ses langues-sœurs est également parlé en Azerbaïdjan, en Russie et en Ouzbékistan (minorité tadjike). Jadis, du XVIe au XIXe siècle, il fut la langue officielle de l'Empire moghol. Le persan fait partie du groupe indo-iranien de la famille des langues indo-européennes. C'est une langue du type « sujet-ob